# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


<module 'gmaps' from 'C:\\Users\\Melvin\\anaconda3\\lib\\site-packages\\gmaps\\__init__.py'>


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the weatherpy csv_file
data_source = pd.read_csv("output_file.csv") 

weather_df = pd.DataFrame(data_source)

weather_df = weather_df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Chokurdakh,RU,2021-01-10 19:51:52,70.6333,147.9167,-30.75,85,99,8.46
1,Hilo,US,2021-01-10 19:50:35,19.7297,-155.0900,75.20,60,1,8.05
2,Nikolskoye,RU,2021-01-10 19:51:52,59.7035,30.7861,6.01,85,100,6.71
3,Ancud,CL,2021-01-10 19:51:52,-41.8697,-73.8203,61.00,50,75,13.80
4,Cururupu,BR,2021-01-10 19:51:52,-1.8283,-44.8683,80.55,79,51,5.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# load up gmaps using the API Key
gmaps.configure(api_key=g_key)


In [43]:
# Store "lat" and "lng" locations from weather_df
locations = weather_df[["Lat", "Lng"]]

# Convert Humidity to Float and store
humidity = weather_df["Humidity (%)"].astype(float)

# Convert Temperature to Float and store
temp_data = weather_df["Max Temp (F)"].astype(float)

In [50]:
figure_layout = {
    "width": "800px",
    "height": "450px",
    "border": "1px solid black",
    "padding": "1px"
}
fig =gmaps.figure(layout=figure_layout)

head_layer =gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(head_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
# find cities that match my ideal based on temperatures above 82 and below 96°F
not_cold = weather_df.loc[weather_df["Max Temp (F)"]> 82]
target_spot = not_cold.loc[weather_df["Max Temp (F)"]< 96]

# reduce the amount of windy conditions
breeze_ezy = target_spot.loc[target_spot["Wind Speed (mph)"]<11]

# look for partly cloudy skies
goldy_spot = breeze_ezy.loc[breeze_ezy["Cloudiness (%)"] <25]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
# make a new dataframe based on city name, temperature and location 
hotel_df = pd.DataFrame(goldy_spot, columns=["City", "Country", "Max Temp (F)", "Lat", "Lng"])

#add an empty column to store hotel names
hotel_df["Hotel"] =""
hotel_df

,City,Country,Max Temp (F),Lat,Lng,Hotel
134,Conde,BR,84.20,-7.2597,-34.9075,
142,Champerico,GT,89.60,14.3000,-91.9167,
143,Marsassoum,SN,82.40,12.8275,-15.9806,
194,Villarrica,PY,89.33,-25.7500,-56.4333,
243,Morón,AR,93.99,-34.6534,-58.6198,
256,Lázaro Cárdenas,MX,82.38,17.9583,-102.2000,
292,Kapoeta,SS,84.04,4.7717,33.5903,


In [72]:
# geocoordinates
target_coordinates = f"{lat},{lng}"
target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

cities = goldy_spot["City"]
lat = []
lng = []

for city in cities:
    lat.append(goldy_spot.loc[:"Lat"])
    lng.append(goldy_spot.loc[:"Lng"])

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    results = response
#     try:
#         Hotel.append(json["results"][0]["name"])
#     except:
#         Hotel.append("")

hotel_df=hotel_df.iloc[hotel_df["Hotel"] = results]

# hotel_df
print(Hotel["results"][0]["name"])

NameError: name 'Hotel' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
